### <font color='blue'>**Data Exploration + XGBoost starter model**
    Pulled from Eric's notebook


In [2]:
import random
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

from xgboost import XGBClassifier
import sklearn
from sklearn.metrics import accuracy_score, precision_recall_curve

import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


modelA2 = XGBClassifier(use_label_encoder=False, max_depth=3)

ModuleNotFoundError: No module named 'xgboost'

In [5]:
class Dataset:
    def __init__(self, pep_df, min_timepoints_per_patient=2):
        '''
        pep_df should be the pep_reduced_intensity_df we created above
        This dataset will only include patients with at least `min_timepoints_per_patient` timepoints
        '''
        self.pep_df = pep_df
        
        self.data = {} # patient id -> [array of timepoint vectors]
        for patient_idx in range(1, 58+1):
            patient_timepoint_vectors = []
            for timepoint in range(1, 7+1):
                try:
                    # We replace all nans with zeros using np.nan_to_num
                    patient_timepoint_vector = np.nan_to_num(self.pep_df[f"Patient_{patient_idx:02d}.Timepoint_{timepoint:01d}"].values)
                    patient_timepoint_vectors.append(patient_timepoint_vector)
                except:
                    # This patient timepoint doesnt exist in the data
                    pass
            
            # If this patient had at least `min_timepoints_per_patient` timepoints, then include this patient id
            if len(patient_timepoint_vectors) >= min_timepoints_per_patient:
                self.data[patient_idx] = patient_timepoint_vectors
                
    @staticmethod
    def prepare_df(filepath):
        df = pd.read_csv(filepath, sep="\t")
        
        # only keep the Patient_x.Timepoint_y formatted columns
        df = df.iloc[:, :667]
        
        # Remove all _unmod columns
        df = df[df.columns.drop(list(df.filter(regex='.\_unmod')))]
        
        # Convert the string abundance numbers into ints for all patient timepoint cols
        def convert_to_number(val):
            if isinstance(val, str):
                return int(val.replace(",","").strip())
            return val

        for patient_timepoint_col in df.columns.values[32:]:
            df[patient_timepoint_col] = df[patient_timepoint_col].apply(convert_to_number)
            df[patient_timepoint_col] = df[patient_timepoint_col].astype(float)
        
        return df
    
    @classmethod
    def from_file(cls, filepath, *args, **kwargs):
        pip_df = cls.prepare_df(filepath)
        return cls(pip_df, *args, **kwargs)
        
                
    def data_generator(self, patient_ids, num_samples, peptide_indices=None, same_patient_pair_probability=0.5):
        """
        This will return a generator that will yield `num_samples` sample pairs of peptide vectors 
        (limited to only the `patient_ids` given, and the `peptide_indices` given (if none is given, then all peptides will be included))
        such that, `same_patient_pair_probability` of the time, the pair of vectors will be from the same patient (but different time points)
                
        Data samples yielded by the returned generator will be of the form:
        {
            "first_patient_idx" : ...,
            "first_patient_timepoint_vector" : ...,
            "second_patient_idx" : ...,
            "second_patient_timepoint_vector" : ...,
            "is_same_patient" : True/False
        }
        
        """
        assert same_patient_pair_probability is not None
        
        # First ensure all of the given patient ids are in out prepared dataset
        assert all(pid in self.data for pid in patient_ids), "Not all of the given patient ids are in our dataset"
        
        def data_gen():
            # This generator is used when same_patient_pair_probability is NOT None
            for _ in range(num_samples):
                
                # pick a random first patient id
                first_patient_id = random.choice(patient_ids)
                
                # pick a random timepoint vector for this first patient id
                first_patient_timepoint_vector = random.choice(self.data[first_patient_id])
                
                # decide if the second patient in our pair should be the same as the first
                is_second_patient_same = random.random() <= same_patient_pair_probability
                
                if is_second_patient_same:
                    second_patient_id = first_patient_id
                else:
                    second_patient_id = random.choice([pid for pid in patient_ids if pid != first_patient_id])
                
                # Note: In this approach, it is possible that, when the second patient = first patient,
                # They both return the same timepoint vector. but that should be okay, since this will help the model learn
                # to detect identical inputs, especially when the input to the model is the concatenated vector
                second_patient_timepoint_vector = random.choice(self.data[second_patient_id])
                
                # if peptide_indices is given, filter the vectors to only include the `peptide_indices` peptides
                if peptide_indices:
                    first_patient_timepoint_vector = first_patient_timepoint_vector[peptide_indices]
                    second_patient_timepoint_vector = second_patient_timepoint_vector[peptide_indices]
                
                yield {
                    "first_patient_id" : first_patient_id,
                    "first_patient_timepoint_vector" : first_patient_timepoint_vector,
                    "second_patient_id" : second_patient_id,
                    "second_patient_timepoint_vector" : second_patient_timepoint_vector,
                    "is_same_patient" : first_patient_id == second_patient_id
                }

        return data_gen()

In [6]:
DATA_DIR = "./Data"
DATA_TSV_FILEPATH = os.path.join(DATA_DIR, "peptidoforms_intensity", "data.tsv")

In [7]:
# Establish all random seeds
random.seed(2021)

# TODO: other libraries should be seeded here, like sklearn, torch, xgboost, etc.

### <font color='blue'>**Creating Main Dataset**

In [8]:
peptidoforms_intensity_dataset = Dataset.from_file(DATA_TSV_FILEPATH)

In [9]:
peptidoforms_intensity_df = peptidoforms_intensity_dataset.pep_df
peptidoforms_intensity_df.shape

(40921, 350)

---

### <font color='blue'>**Creating Train/Val/Test Datasets**
    
Train/val/test datasets will enforce the `same_patient_pair_probability = 0.5`

In [10]:
train_ratio = 0.6
val_ratio = 0.2
test_ratio = 0.2

In [11]:
# For now, we don't select a particular subset of peptides, and just use all of them.
selected_peptide_indices = None

In [12]:
train_same_patient_pair_probability = 0.5
val_same_patient_pair_probability = 0.5
test_same_patient_pair_probability = 0.5

In [13]:
# using 1k samples for train/val/test finishes fast, but may not be accurate
# so we should train with more samples (~100K train/val/test) with colab/gpu later, which would take more time
num_train_samples = 10000
num_val_samples = 10000
num_test_samples = 10000

In [14]:
all_patient_ids = list(peptidoforms_intensity_dataset.data.keys())
num_patient_ids = len(all_patient_ids)
num_patient_ids

52

In [15]:
random.shuffle(all_patient_ids)
train_patient_ids, val_patient_ids, test_patient_ids = np.split(all_patient_ids, 
                                                                [int(train_ratio * num_patient_ids), 
                                                                 int((train_ratio + val_ratio) * num_patient_ids)
                                                                ])
train_patient_ids = train_patient_ids.tolist()
val_patient_ids = val_patient_ids.tolist()
test_patient_ids = test_patient_ids.tolist()

len(train_patient_ids), len(val_patient_ids), len(test_patient_ids)

(31, 10, 11)

In [16]:
def create_data_arrays_from_generator(gen):
    """
    returns (concatX, absdiffX Y)
    
    concatX is the two input feature vectors concatenated together
    absdiffX is the abs difference between the two input feature vectors
    Y is always 1/0 if the vectors are from the same patient or not
    """    
    concatX, absdiffX, Y = [], [], []
    
    for data_input in tqdm(gen):
        concatX.append(np.hstack((data_input['first_patient_timepoint_vector'], data_input['second_patient_timepoint_vector'])).astype(int))
        absdiffX.append(np.abs(data_input['first_patient_timepoint_vector'] - data_input['second_patient_timepoint_vector']).astype(int))
        Y.append(1 if data_input['is_same_patient'] else 0)
    
    return np.array(concatX), np.array(absdiffX), np.array(Y)

In [17]:
train_data_gen = peptidoforms_intensity_dataset.data_generator(
    patient_ids=train_patient_ids, 
    num_samples=num_train_samples, 
    peptide_indices=selected_peptide_indices, 
    same_patient_pair_probability=train_same_patient_pair_probability)
concatTrainX, absDiffTrainX, trainY = create_data_arrays_from_generator(train_data_gen)

val_data_gen = peptidoforms_intensity_dataset.data_generator(
    patient_ids=val_patient_ids, 
    num_samples=num_val_samples, 
    peptide_indices=selected_peptide_indices, 
    same_patient_pair_probability=val_same_patient_pair_probability)
concatValX, absDiffValX, valY = create_data_arrays_from_generator(val_data_gen)

test_data_gen = peptidoforms_intensity_dataset.data_generator(
    patient_ids=test_patient_ids, 
    num_samples=num_test_samples, 
    peptide_indices=selected_peptide_indices, 
    same_patient_pair_probability=test_same_patient_pair_probability)
concatTestX, absDiffTestX, testY = create_data_arrays_from_generator(test_data_gen)

10000it [00:10, 979.81it/s]
10000it [00:10, 951.29it/s]
10000it [00:09, 1064.39it/s]


In [18]:
# Sanity check that the labels are in a rough 50-50 distribution
pd.Series(trainY).value_counts()

0    5005
1    4995
dtype: int64

In [19]:
# Sanity check that the labels are in a rough 50-50 distribution
pd.Series(valY).value_counts()

1    5079
0    4921
dtype: int64

In [20]:
# Sanity check that the labels are in a rough 50-50 distribution
pd.Series(testY).value_counts()

1    5033
0    4967
dtype: int64

---

### <font color='blue'>**XGBoost model variants**</font>

The different variants we will be training are the following:

First, train 2 models where one takes in concatenated features vs. the other takes in the absolute value difference of the input feature vectors. Let the better of these 2 models (based on the lowest val loss achieved) be called model A.

Using model A, get its important feature weights, and try to train the same model architecture but only using the top k features (where k = 1, 10, 100, num_nonzeroweight_peptides_from_modelA).

Finally, using model A, also train only using peptides present in at most N patients for N = (5, 10, 20, 40).

---

**Sanity check:** Initial log loss should be around $0.69$, since $-ln(0.5) = 0.6931471806$ and there is a $0.5$ probability of same/different patient

In [21]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_recall_curve

In [52]:
modelA1 = XGBClassifier(use_label_encoder=False, max_depth=3)
modelA2 = XGBClassifier(use_label_encoder=False, max_depth=3)

XGBoostError: sklearn needs to be installed in order to use this module

In [ ]:
# trained on concatenated vectors
print("Training model A1: concatenated input feature vectors")
modelA1.fit(concatTrainX, trainY, eval_metric="logloss", eval_set=[(concatValX, valY)], verbose=True)

print("\n\n")

# trained on concatenated vectors
print("Training model A2: abs difference of input feature vectors")
modelA2.fit(absDiffTrainX, trainY, eval_metric="logloss", eval_set=[(absDiffValX, valY)], verbose=True)

In [ ]:
## Save models
modelA1.save_model('xgboost_modelA1.json')
modelA2.save_model('xgboost_modelA2.json')

### Model A variants - metrics

In [ ]:
import matplotlib.pyplot as plt

In [ ]:


# helper method to print basic model metrics
def metrics(y_true, y_pred):
    print('Confusion matrix:\n', confusion_matrix(y_true, y_pred))
    print('\nReport:\n', classification_report(y_true, y_pred))

In [ ]:
def print_model_metrics(model, model_input_format, mode, peptide_indices=None):
    assert model_input_format in ("concat", "abs_diff")
    assert mode in ("train", "val", "test")
    
    if peptide_indices is None:
        x_s = {
            "train" : {
                "concat" : concatTrainX,
                "abs_diff" : absDiffTrainX
            },
            "val" : {
                "concat" : concatValX,
                "abs_diff" : absDiffValX
            },
            "test" : {
                "concat" : concatTestX,
                "abs_diff" : absDiffTestX
            }
        }
    else:
        x_s = {
            "train" : {
                "concat" : concatTrainX,
                "abs_diff" : absDiffTrainX[:,peptide_indices]
            },
            "val" : {
                "concat" : concatValX,
                "abs_diff" : absDiffValX[:,peptide_indices]
            },
            "test" : {
                "concat" : concatTestX,
                "abs_diff" : absDiffTestX[:,peptide_indices]
            }
        }


    y_s = {
        "train" : trainY,
        "val" : valY,
        "test" : testY
    }
    
    x = x_s[mode][model_input_format]
    y = y_s[mode]
        
    pred_scores = model.predict(x)
    preds = [round(pred_score) for pred_score in pred_scores]
    
    metrics(y, preds)
    acc = accuracy_score(y, preds)
    
    print (f"{mode} accuracy: {acc}")
    
    precision, recall, thresholds = precision_recall_curve(y, pred_scores)
    
    # plotting below taken from: 
    # https://machinelearningmastery.com/roc-curves-and-precision-recall-curves-for-imbalanced-classification/
    
    # calculate the no skill line as the proportion of the positive class
    no_skill = len(y[y==0]) / len(y)
    
    # plot the no skill precision-recall curve
    plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='Trivial model')
    
    # plot the model precision-recall curve
    plt.plot(recall, precision, marker='.', label='XGBoost model')
    
    # axis labels
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    
    # show the legend
    plt.legend()
    
    # show the plot
    plt.show()

#### Model A1

In [ ]:
print_model_metrics(modelA1, "concat", "train")

In [ ]:
print_model_metrics(modelA1, "concat", "val")

In [ ]:
print_model_metrics(modelA1, "concat", "test")

#### Model A2

In [ ]:
print_model_metrics(modelA2, "abs_diff", "train")

In [ ]:
print_model_metrics(modelA2, "abs_diff", "val")

In [ ]:
print_model_metrics(modelA2, "abs_diff", "test")

### Training model A2 using only the top k peptides (k = 1, 5, 10, 25, 50, num_peptides_with_nonzero_weight)

In [ ]:
ft = modelA2.feature_importances_
np.save("modelA2_feature_importances.npy", ft)

In [ ]:
num_peptides_with_nonzero_weight = len(ft[ft > 0]) # count number of peptides with a non-zero weight
num_peptides_with_nonzero_weight

In [ ]:
num_unique_peptides = len(peptidoforms_intensity_df)
assert num_unique_peptides == len(ft)

In [ ]:
top_peptidoform_indices = ft.argsort()[::-1]

In [ ]:
# lets get some of the most important (top 10) peptide indices
top_peptidoform_indices[:10]

In [ ]:
# weights of the top 10 most important peptides
ft[top_peptidoform_indices[:10]]

In [ ]:
def isolate_peptide_indices(X, peptide_indices):
    # use only the given peptide indices from the given X matrix, where X is a list of all the absolute difference vectors
    # since this model format performed the best, when compared to the concatenated vector representation
    return X[:,peptide_indices]

In [ ]:
def try_model_on_top_k_peptides(k_values=(1, 5, 10, 25, 50, num_peptides_with_nonzero_weight)):
    
    topkpeptide_models = []
    for k in k_values:
        print ("="*50)
        print (f"K value: {k}")
        print ("="*50)
        
        k_trainX = isolate_peptide_indices(absDiffTrainX, top_peptidoform_indices[:k])
        k_trainY = trainY
        
        k_valX = isolate_peptide_indices(absDiffValX, top_peptidoform_indices[:k])
        k_valY = valY
        
        topk_model = XGBClassifier(use_label_encoder=False, max_depth=3)
        topk_model.fit(k_trainX, k_trainY, eval_metric="logloss", eval_set=[(k_valX, k_valY)], verbose=True)
        topk_model.save_model(f"xgboost_modelA2_top{k}_peptides.json")
        topkpeptide_models.append(topk_model)
        
        print_model_metrics(topk_model, "abs_diff", "train", peptide_indices=top_peptidoform_indices[:k])
        print_model_metrics(topk_model, "abs_diff", "val", peptide_indices=top_peptidoform_indices[:k])
        print_model_metrics(topk_model, "abs_diff", "test", peptide_indices=top_peptidoform_indices[:k])

        print ("\n\n")
    
    return topkpeptide_models

In [ ]:
_ = try_model_on_top_k_peptides()

### <font color='blue'>**Important peptides analysis**

In [ ]:
# x-axis is weight and y-axis is count
# this shows that very, very few peptides have a significant weight (ie., > 0.035)
pd.Series(ft[ft > 0]).hist()

In [ ]:
# Because the first half of the features and the second half are semantically the same, 
# we have to % by the number of unique peptides (ie., the size of the first half of each combined feature vector)
# to get the unique set of peptides that are deemed important
important_peptide_indices = np.where(ft > 0)[0]
len(important_peptide_indices)

In [ ]:
def get_patients_peptide_is_present_in(peptide_index):
    present_patient_timepoints = peptidoforms_intensity_df.iloc[peptide_index, 32:].dropna().index
    
    patients = set()
    for pt in present_patient_timepoints:
        spltidx = pt.index(".Timepoint_")
        patient_id = int(pt[:spltidx].replace("Patient_", ""))
        patients.add(patient_id)
    
    return patients

In [ ]:
for peptide_index in top_k_peptidoform_indices:
    print (f"Peptide {peptide_index}, weight: {ft[peptide_index]}")
    print (f"{len(get_patients_peptide_is_present_in(peptide_index))} patients have this peptide")
    print("\n")

In [ ]:
original_data_df = pd.read_csv(DATA_TSV_FILEPATH, sep="\t")
assert original_data_df.shape[0] == peptidoforms_intensity_df.shape[0]

In [ ]:
original_data_df = original_data_df.iloc[important_peptide_indices]
original_data_df['model_weight'] = ft[important_peptide_indices]

In [ ]:
original_data_df.to_csv("important_peptidoforms_selected_by_xgboost_all_columns.csv", index=None)

In [ ]:
export_peptidoforms_intensity_df = peptidoforms_intensity_df.iloc[important_peptide_indices]
export_peptidoforms_intensity_df['model_weight'] = ft[important_peptide_indices]
export_peptidoforms_intensity_df.to_csv("important_peptidoforms_selected_by_xgboost.csv", index=None)

## TODO: Any necessary code for Results section

In [ ]:
top_peptidoform_indices_and_weights = list(zip(important_peptide_indices, ft[important_peptide_indices]))
sorted(top_peptidoform_indices_and_weights, key = lambda item: item[1], reverse=True)